# CECS 229 Coding Assignment #6

#### Due Date: 

Sunday, 4/24 @ 11:59 PM

#### Submission Instructions:

To receive credit for this assignment you must submit the following by the due date:

1. **To the BB Dropbox Folder:** this completed .ipynb file

2. **To CodePost:** this file converted to a Python script named `ca6.py`

#### Objectives:

1. Apply Gaussian Elimination, with and without pivoting, to help solve the system $A \overrightarrow{x} = \overrightarrow{b}$.
2. Use Lp -norm to calculate the error in a solution given by applying any of the Gaussian elimination algorithms.
3. Use the RREF of the augmented matrix for the system $A \overrightarrow{x} = \overrightarrow{b}$ to determine if it has one solution, no solution, or infinitely-many solutions.
4. Determine the number of free variables that the system $A \overrightarrow{x} = \overrightarrow{b}$ has if it has infinitely-many solutions.
5. Determine whether a set of column-vectors is linearly dependent by forming and solving a system of the form $A \overrightarrow{x} = \overrightarrow{0}$.
-----------------------------------------



#### Problem 1

Copy-paste your implemented `Matrix` and `Vec` classes to the next cell.  Then, complete the following tasks:

1. Add a method `norm(p)` to your `Vec` class so that if `u` is a `Vec` object, then `u.norm(p)` returns the $L_p$-norm of vector `u`.  Recall that the $L_p$-norm of an $n$-dimensional vector $\overrightarrow{u}$ is given by, $||u||_p = \left( \sum_{i = 1}^{n} |u_i|^p\right)^{1/p}$.  Input `p` should be of the type `int`.  The output norm should be of the type `float`.
2. Add a method `rank()` to your `Matrix` class so that if `A` is a `Matrix` object, then `A.rank()` returns the rank of `A`.

In [1]:
import math
class Vec:
    def __init__(self, contents = []):
        """
        Constructor defaults to empty vector
        INPUT: list of elements to initialize a vector object, defaults to empty list
        """
        self.elements = contents
        return 
    
    def __abs__(self):
        """
        Overloads the built-in function abs(v)
        returns the Euclidean norm of vector v
        """
        return math.sqrt(sum([i**2 for i in self.elements]))

        
    def __add__(self, other):
        """Overloads the + operator to support Vec + Vec
         raises ValueError if vectors are not same length
        """
        if len(self.elements) != len(other.elements):
            raise ValueError
        else:
            return Vec([x + y for x, y in zip(self.elements, other.elements)])


    
    def __sub__(self, other):
        """
        Overloads the - operator to support Vec - Vec
        Raises a ValueError if the lengths of both Vec objects are not the same
        """
        if len(self.elements) != len(other.elements):
            raise ValueError
        else:
            return Vec([x-y for x,y in zip(self.elements,other.elements)])

    
    
    def __mul__(self, other):
        """Overloads the * operator to support 
            - Vec * Vec (dot product) raises ValueError if vectors are not same length in the case of dot product
            - Vec * float (component-wise product)
            - Vec * int (component-wise product)
        """
        if type(other) == Vec:
            if len(self.elements) != len(other.elements):
                raise ValueError
            elif len(self.elements) == len(other.elements):
                return (sum([x*y for x,y in zip(self.elements,other.elements)]))
        elif type(other) == float or type(other) == int: #scalar-vector multiplication
            return Vec([i*other for i in self.elements])
            
    
    def __rmul__(self, other):
        """Overloads the * operation to support 
            - float * Vec
            - int * Vec
        """
        return Vec([i * other for i in self.elements])
    

    
    def __str__(self):
        """returns string representation of this Vec object"""
        return str(self.elements) # does NOT need further implementation

    def norm(self,p):
        y = 0
        for i in self.elements:
            x=abs(i)**p
            y+=x
        return y**(1/p)

"----------------------------------------------------------------------------------------------------------------------"

class Matrix:
    
    def __init__(self,rows = []):  
        if len(rows) != 0:
            cols = len(rows[0])
            if cols == 0:
                raise ValueError
            for row in rows:
                if len(row) != cols:
                    raise ValueError
                for value in row:
                    if not isinstance(value,(int,float)):
                        raise ValueError
            self.rows = rows
        else:
            self.rows = []


    def set_col(self, j, u):
        if len(u) != len(self.rows):
            raise ValueError("Incompatible column length.")    
        else:
            self.rows  = [self.rows[i][0:j-1] + [u[i]] + self.rows[i][j:] for i in range(len(self.rows))]
        return self


    def set_row(self,i, v):
        if len(v) != len(self.rows[0]):
            raise ValueError("Incompatible row length.")    
        else:
            self.rows[i-1] = v
        return self

    def set_entry(self,i, j, x):
        self.rows[i-1][j-1] = x
        return self
        
    
    def get_col(self,j):
        return [self.rows[i][j-1] for i  in range(len(self.rows))]

    
    def get_row(self,i):
        return self.rows[i-1]

    
    def get_entry(self, i ,j):
        return self.rows[i-1][j-1]
    

    def col_space(self):
        return[[row[i] for row in self.rows] for i in range(len(self.rows[0]))]
    

    def row_space(self):
        return self.rows

    
    def get_diag(self, k):
        if k == 0:
            return [self.rows[i][i] for i in range(len(self.rows))]
        if k > 0:
            return[self.rows[i][i+(abs(k))] for i in range(len(self.rows)-abs(k))]
        if k < 0:
             return[self.rows[i+(abs(k))][i] for i in range(len(self.rows)-abs(k))]

        

    
    def __str__(self):
        if len(self.rows) == 0:
            return "[]"
        if len(self.rows) == 1:
            return "[[" + ", " .join(self.rows[0]) + "]]"
        return("" + "\n".join([
            "[" + ", ".join([str(value) for value in row]) + "]"
            for row in self.rows
        ])
        + "")




    def __add__(self, other):
        if ((len(self.rows) != len(other.rows)) or (len(self.rows[0])!= len(other.rows[0]))):
            raise ValueError("Matrix must be of the same size.")
        return Matrix(
            [[self.rows[i][j] + other.rows[i][j] for j in range((len(self.rows[0])))]
            for i in range(len(self.rows))]
        )
        
    def __sub__(self, other):
        if ((len(self.rows) != len(other.rows)) or (len(self.rows[0])!= len(other.rows[0]))):
            raise ValueError("Matrix must be of the same size.")
        return Matrix(
            [[self.rows[i][j] - other.rows[i][j] for j in range((len(self.rows[0])))]
            for i in range(len(self.rows))]
        )
        
    def __mul__(self, other):  
        if type(other) == float or type(other == int):
            return Matrix([[item * other for item in row] for row in self.rows])
        elif type(other) == Matrix:
            if ((len(self.rows[0]) != len(other.rows))):
                raise ValueError("Matrix must be the same size")
            return Matrix([
                [sum(row[i] * column[i] for i in range(len(row)))for column in [[row[i] for row in other.rows] 
                for i in range(len(other.rows[0]))]]
                for row in self.rows
                ]
            )
        elif type(other) == Vec:
            if len(self.rows[0]) != len(other.elements):
                raise ValueError("Vector must be of length of columns.")
            result = []
            for i in range(len(self.rows[0])): 
                total = 0
                for j in range(len(other.elements)): 
                    total += other.elements[j] * self.rows[i][j]
                result.append(total)
            return Vec(result)

        else:
            raise TypeError("ERROR: Unsupported Type.")
    
    def __rmul__(self, other):  
        if type(other) == float or type(other) == int:
           return Matrix([[item * other for item in row] for row in self.rows])
        else:
            raise TypeError("ERROR: Unsupported Type.")
        
        
    def __eq__(self, other):
        """overloads the == operator to return True if 
        two Matrix objects have the same row space and column space"""
        this_rows = self.row_space()
        other_rows = other.row_space()
        this_cols = self.col_space()
        other_cols = other.col_space()
        return this_rows == other_rows and this_cols == other_cols

    def __req__(self, other):
        """overloads the == operator to return True if 
        two Matrix objects have the same row space and column space"""
        this_rows = self.row_space()
        other_rows = other.row_space()
        this_cols = self.col_space()
        other_cols = other.col_space()
        return this_rows == other_rows and this_cols == other_cols
    
    def rank(self):
        r1 = len(self.rows)
        rank = len(self.rows[0])
        for r in range(0,rank,1):
            if self.rows[r][r] != 0:
                for c in range(0,r1,1):
                    if c != r:
                        m = (self.rows[c][r] / self.rows[r][r])
                        for i in range(rank):
                            self.rows[c][i] -= (m * self.rows[r][i])
            else:
                shrink = True
                for i in range(r+1,r1,1):
                    if self.rows[i][r] != 0:
                        for b in range(c):
                            temp = self.rows[r][b]
                            self.rows[r][b] = self.rows[i][b]
                            self.rows[i][b] = temp
                        shrink = False
                        break
                if shrink:
                    rank -=1
                    for i in range(0,r1,1):
                        self.rows[i][r] = self.rows[i][rank]
                r-=1
        return rank

#### Problem 2

1. Implement a helper function called `_rref(A, b)` that applies Gaussian Elimination ***without pivoting*** to return the Reduced-Row Echelon Form of the augmented matrix formed from `Matrix` object `A` and `Vec` object `b`.  The output must be of the type `Matrix`.
2. Implement the function `solve_np(A, b)` that uses `_rref(A)` to solve the system $A \overrightarrow{x} = \overrightarrow{b}$.  The input `A` is of the type `Matrix` and `b` is of the type `Vec`.
    - If the system has a unique solution, it returns the solution as a `Vec` object.  
    - If the system has no solution, it returns `None`. 
    - If the system has infinitely many solutions, it returns the number of free variables (`int`) in the solution.

In [32]:
def _rref(A, b):
    augmented = Matrix(A.get_row())
def solve_np(A, b):
    pass

#### Problem 3

1. Implement a helper function called `_rref_pp(A, b)` that applies Gaussian Elimination ***with partial pivoting*** to return the Reduced-Row Echelon Form of the augmented matrix formed from `Matrix` object `A` and `Vec` object `b`.  The output must be of the type `Matrix`.
2. Implement the function `solve_pp(A, b)` that uses `_rref_pp(A, b)` to solve the system $A \overrightarrow{x} = \overrightarrow{b}$.  The input `A` is of the type `Matrix` and `b` is of the type `Vec`.  
    - If the system has a unique solution, it returns the solution as a `Vec` object.  
    - If the system has no solution, it returns `None`. 
    - If the system has infinitely many solutions, it returns the number of free variables (`int`) in the solution.

In [ ]:
def _rref_pp(A, b):
    # todo
    # find largest number move it then call _rref 
    pass

def solve_pp(A, b):
    #todo
    #same as the other 
    pass

#### Problem 4

1. Implement a helper function called `_rref_tp(A, b)` that applies Gaussian Elimination ***with total pivoting*** to return the Reduced-Row Echelon Form of the augmented matrix formed from `Matrix` object `A` and `Vec` object `b`.  The output must be of the type `Matrix`. 
2. Implement the function `solve_tp(A, b)` that uses `_rref_tp(A)` to solve the system $A \overrightarrow{x} = \overrightarrow{b}$.  The input `A` is of the type `Matrix` and `b` is of the type `Vec`. 
    - If the system has a unique solution, it returns the solution as a `Vec` object.  
    - If the system has no solution, it returns `None`. 
    - If the system has infinitely many solutions, it returns the number of free variables (`int`) in the solution.

In [ ]:
def _rref_tp(A):
    # todo
    # call _rref_pp then find the largest number, move it, then call _rref
    pass
def solve_tp(A, b):
    #todo
    #same as the other 
    pass

In [ ]:
from numpy import array, zeros, fabs, linalg

a = array([[0, 6, -1, 2, 2],
           [0, 3, 4, 1, 7],
           [5, 1, 0, 3, -1],
           [3, 1, 3, 0, 2],
           [4, 4, 1, -2, 1]], float)
#the b matrix constant terms of the equations 
b = array([5, 7, 2, 3, 4], float)

print("Solution by NumPy:")


print(linalg.solve(a, b))

n = len(b)
x = zeros(n, float)

#first loop specifys the fixed row
for k in range(n-1):
    if fabs(a[k,k]) < 1.0e-12:
        
        for i in range(k+1, n):
            if fabs(a[i,k]) > fabs(a[k,k]):
                a[[k,i]] = a[[i,k]]
                b[[k,i]] = b[[i,k]]
                break

 #applies the elimination below the fixed row

    for i in range(k+1,n):
        if a[i,k] == 0:continue

        factor = a[k,k]/a[i,k]
        for j in range(k,n):
            a[i,j] = a[k,j] - a[i,j]*factor
            #we also calculate the b vector of each row
        b[i] = b[k] - b[i]*factor
print(a)
print(b)


x[n-1] = b[n-1] / a[n-1, n-1]
for i in range(n-2, -1, -1):
    sum_ax = 0
  
    for j in range(i+1, n):
        sum_ax += a[i,j] * x[j]
        
    x[i] = (b[i] - sum_ax) / a[i,i]

print("The solution of the system is:")
print(x)

#### Master function

The following function is the master function that will be called by the CodePost tester.  It will be fully functional once you have completed Problems 1 - 4.  No edits are necessary.

In [2]:
import enum
class GaussSolvers(enum.Enum):
    np = 0
    pp = 1
    tp = 2
    
    
def solve(A, b, solver = GaussSolvers.np):
    if solver == GaussSolvers.np:
        return solve_np(A, b)
    elif solver == GaussSolvers.pp:
        return solve_pp(A, b)
    elif solver == GaussSolvers.tp:
        return solve_tp(A, b)

In [ ]:
"""TESTER CELL #1"""
A = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

x = Vec([2.3, -4.1, 5.7]) # this is the true solution

b = A * x

x_np = solve(A, b)
x_pp = solve(A, b, GaussSolvers.pp)
x_tp = solve(A, b, GaussSolvers.tp)

epsilon_np = x_np - x
epsilon_pp = x_pp - x
epsilon_tp = x_tp - x

error1_np = epsilon_np.norm(1)
error2_np = epsilon_np.norm(2)

print("-"*20)
print("No Pivoting Solution:", x_np)
print("True solution:", x)

print("Errors in Gaussian Elimination Without Pivoting")
print("L1-norm error:", error1_np)
print("L2-norm error:", error1_np)
print()

print("-"*20)
print("Partial Pivoting Solution:", x_pp)
print("True solution:", x)

error1_pp = epsilon_pp.norm(1)
error2_pp = epsilon_pp.norm(2)

print("Errors in Gaussian Elimination With Partial Pivoting")
print("L1-norm error:", error1_pp)
print("L2-norm error:", error1_pp)
print()

print("-"*20)
print("Total Pivoting Solution:", x_tp)
print("True solution:", x)

error1_tp = epsilon_tp.norm(1)
error2_tp = epsilon_tp.norm(2)

print("Errors in Gaussian Elimination With Total Pivoting")
print("L1-norm error:", error1_tp)
print("L2-norm error:", error1_tp)


In [ ]:
"""TESTER CELL #2"""

A = Matrix([[1, 2, 3], [2, 4, 6]])

b = Vec([6, -12]) 

x_np = solve(A, b)
x_pp = solve(A, b, GaussSolvers.pp)
x_tp = solve(A, b, GaussSolvers.tp)

print("-"*20)
print("No Pivoting Solution:", x_np)
print("Expected: None")

print("-"*20)
print("Partial Pivoting Solution:", x_pp)
print("Expected: None")

print("-"*20)
print("Total Pivoting Solution:", x_tp)
print("Expected: None")

In [ ]:
"""TESTER CELL #3"""

# Test one of the examples from lecture that had infinitely-many solutions


------------------------------------------------

#### Problem 5

Implement the method `is_independent(S)` that returns `True` if the set `S` of `Vec` objects is linearly **independent**, otherwise returns `False`.

In [ ]:
def is_independent(S):
    pass
    

In [ ]:
"""TESTER CELL"""

S1 = {Vec([1, 2]), Vec([2, 3]), Vec([3, 4])}
print("S1 is Independent:", is_independent(S1))
print("Expected: False")

S2 = {Vec([1, 1, 1]), Vec([1, 2, 3]), Vec([1, 3, 6])}
print("S2 is Independent:", is_independent(S2))
print("Expected: True")

